In [1]:
from warnings import simplefilter
simplefilter(action='ignore', category=FutureWarning)

import pandas as pd
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score, cross_val_predict, StratifiedKFold, train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, classification_report, multilabel_confusion_matrix
import tensorflow as tf
import datetime, os
from tensorflow.keras.layers import Input, Dense, Activation, Dropout, Flatten, concatenate
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import ModelCheckpoint
from tensorflow.keras.utils import plot_model
from tensorflow.keras.optimizers import SGD
from tensorflow.keras.constraints import MaxNorm
from sklearn.model_selection import GridSearchCV
from splitrepeat import splitrepeat_cv

config = tf.compat.v1.ConfigProto(gpu_options = tf.compat.v1.GPUOptions(per_process_gpu_memory_fraction=0.8))
config.gpu_options.allow_growth = True
session = tf.compat.v1.Session(config=config)
tf.compat.v1.keras.backend.set_session(session)

features=['moca_digits','moca_letters','moca_serial7','MMSPELL_early','MMSPELL_late','ADAS_Q1','moca_orient','ADAS_Q7','ADAS_Q8','ADAS_Q9','MMDATE',
       'MMYEAR', 'MMMONTH', 'MMDAY', 'MMSEASON', 'MMHOSPIT', 'MMFLOOR','MMCITY', 'MMAREA', 'MMSTATE','nbspan_forward',
       'faq7','faq8','moca_recall','ADAS_Q4','MMBALLDL', 'MMFLAGDL', 'MMTREEDL', 'LDELTOTAL', 'AVRECALL', 'AVDEL30MIN', 'AVDELTOT', 'AVDELERR2',
       'faq9','moca_clock','MMDRAW','CLOCKCIRC', 'CLOCKSYM','CLOCKNUM', 'CLOCKHAND', 'CLOCKTIME', 'COPYCIRC', 'COPYSYM',
       'COPYNUM', 'COPYHAND', 'COPYTIME', 'tmab_time', 'TRAAERRCOM','TRAAERROM', 'TRABERRCOM', 'TRABERROM','moca_naming','moca_repeat','moca_fluency',
       'moca_similarities','ADAS_Q2', 'ADAS_Q5','ADAS_Q10','ADAS_Q11','ADAS_Q12','MMBALL', 'MMFLAG', 'MMTREE',
       'MMWATCH', 'MMPENCIL','MMREPEAT','MMREAD', 'MMWRITE', 'CATANIMSC', 'CATVEGESC', 'moca_visuo_exec','ADAS_Q3','ADAS_Q6','ADAS_Q13','MMHAND', 'MMFOLD',
       'MMONFLR','nbspan_backward','faq1','faq2','faq3', 'faq4','faq5','faq6','faq10','PXGENAPP', 'PXHEADEY', 'PXNECK', 'PXCHEST',
       'PXHEART', 'PXABDOM', 'PXEXTREM', 'PXPERIPH', 'PXSKIN', 'PXMUSCUL', 'e_memory_pt', 'e_lang_pt', 'e_visspat_pt', 'e_plan_pt', 'e_organ_pt',
       'e_divatt_pt','e_memory_cg', 'e_lang_cg', 'e_visspat_cg','e_plan_cg', 'e_organ_cg', 'e_divatt_cg']


categorical_features = np.in1d(features, ['MMONFLR', 'faq10', 'faq6', 'COPYSYM', 'COPYNUM', 'faq5', 'MMTREE', 'COPYHAND', 'moca_clock', 'MMSEASON', 'moca_letters', 
                                          'MMBALL', 'faq9', 'MMFLOOR', 'MMDRAW', 'MMMONTH', 'PXGENAPP', 'MMWATCH', 'CLOCKCIRC', 'faq8', 'MMHOSPIT', 'moca_naming', 
                                          'PXEXTREM', 'CLOCKNUM', 'PXMUSCUL', 'faq1', 'MMTREEDL', 'CLOCKTIME', 'PXABDOM', 'MMFLAG', 'COPYCIRC', 'MMAREA', 'faq3', 
                                          'moca_digits', 'CLOCKHAND', 'MMREAD', 'MMYEAR', 'MMREPEAT', 'moca_visuo_exec','MMHAND', 'MMBALLDL', 'PXCHEST', 'MMDATE', 
                                          'MMFLAGDL',  'MMSTATE','moca_repeat', 'MMFOLD', 'MMPENCIL', 'MMDAY',  'faq2', 'PXHEART', 'CLOCKSYM', 'faq4', 'moca_serial7', 
                                          'faq7', 'MMCITY', 'PXHEADEY', 'COPYTIME', 'PXPERIPH', 'PXSKIN', 'moca_fluency', 'moca_similarities', 'PXNECK', 'MMWRITE',
                                          'e_memory_pt', 'e_lang_pt', 'e_visspat_pt', 'e_plan_pt', 'e_organ_pt', 'e_divatt_pt','e_memory_cg', 'e_lang_cg', 
                                          'e_visspat_cg','e_plan_cg', 'e_organ_cg', 'e_divatt_cg'])

data = pd.read_csv('../data/interim/data_adni.csv')
X = data[features]
y = data['CDGLOBAL']
y.replace({1:0, 2:1},inplace=True)

In [2]:
# Divide data in half for tuning, to reduce leakage when using split/repeat cross-validation
X, X_test, y, y_test = train_test_split(X, y, train_size=0.25, random_state=33433, stratify=y)

from imblearn.over_sampling import SMOTENC
sm = SMOTENC(categorical_features=categorical_features, sampling_strategy=.5, random_state=33433)
X, y = sm.fit_resample(X,y)
y = pd.Series(y)   

X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.25, random_state=33433, stratify=y)     

# RF Classifier

In [36]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100, max_features=.5, random_state=0)

%time \
df = splitrepeat_cv(X,y,rf,splits=[111,222,333,444],repeats=[111,222,333,444,555])
df.describe()

Wall time: 7.69 s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.978804,0.952717,0.970109,0.965761,0.952717,0.978804,0.970109,0.965761,0.952717,0.978804,0.970109,0.965761
std,0.011277,0.019081,0.005744,0.007130,0.019081,0.011277,0.005744,0.007130,0.019081,0.011277,0.005744,0.007130
min,0.961957,0.923913,0.960145,0.956522,0.923913,0.961957,0.960145,0.956522,0.923913,0.961957,0.960145,0.956522
25%,0.967391,0.932065,0.967391,0.959239,0.932065,0.967391,0.967391,0.959239,0.932065,0.967391,0.967391,0.959239
50%,0.980978,0.967391,0.967391,0.967391,0.967391,0.980978,0.967391,0.967391,0.967391,0.980978,0.967391,0.967391
75%,0.989130,0.967391,0.971014,0.968071,0.967391,0.989130,0.971014,0.968071,0.967391,0.989130,0.971014,0.968071
max,0.994565,0.967391,0.981884,0.978261,0.967391,0.994565,0.981884,0.978261,0.967391,0.994565,0.981884,0.978261


## Random Search

In [37]:
from sklearn.model_selection import RandomizedSearchCV

# Search parameters
n_estimators = [int(x) for x in np.linspace(start = 200, stop = 2000, num = 10)]
max_features = ['sqrt', 'log2', .3, .5, .7, .9]
max_depth = [int(x) for x in np.linspace(10, 110, num = 11)]
max_depth.append(None)
min_samples_split = [2, 5, 10]
min_samples_leaf = [1, 2, 4]
bootstrap = [True, False]

search_params = {'n_estimators': n_estimators,
               'max_features': max_features,
               'max_depth': max_depth,
               'min_samples_split': min_samples_split,
               'min_samples_leaf': min_samples_leaf,
               'bootstrap': bootstrap}

In [38]:
rf_search_rand = RandomizedSearchCV(rf, search_params, n_iter=400,cv=3,verbose=2,random_state=33433, n_jobs=-1)
rf_search_rand.fit(X_train, y_train)

Fitting 3 folds for each of 400 candidates, totalling 1200 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:   13.9s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  1.2min
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  2.8min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  4.9min
[Parallel(n_jobs=-1)]: Done 989 tasks      | elapsed:  7.4min
[Parallel(n_jobs=-1)]: Done 1200 out of 1200 | elapsed:  8.9min finished


RandomizedSearchCV(cv=3, error_score=nan,
                   estimator=RandomForestClassifier(bootstrap=True,
                                                    ccp_alpha=0.0,
                                                    class_weight=None,
                                                    criterion='gini',
                                                    max_depth=None,
                                                    max_features=0.5,
                                                    max_leaf_nodes=None,
                                                    max_samples=None,
                                                    min_impurity_decrease=0.0,
                                                    min_impurity_split=None,
                                                    min_samples_leaf=1,
                                                    min_samples_split=2,
                                                    min_weight_fraction_leaf=0.0,
                  

In [39]:
rf_search_rand.best_params_

{'n_estimators': 1600,
 'min_samples_split': 5,
 'min_samples_leaf': 2,
 'max_features': 'log2',
 'max_depth': 40,
 'bootstrap': False}

In [40]:
best = rf_search_rand.best_estimator_
df_rand = splitrepeat_cv(X,y,best,splits=[10,20,30],repeats=[10,20,30])
df_rand.describe()

,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.984903,0.967391,0.979066,0.976147,0.967391,0.984903,0.979066,0.976147,0.967391,0.984903,0.979066,0.976147
std,0.003623,0.018827,0.007196,0.009964,0.018827,0.003623,0.007196,0.009964,0.018827,0.003623,0.007196,0.009964
min,0.978261,0.945652,0.967391,0.961957,0.945652,0.978261,0.967391,0.961957,0.945652,0.978261,0.967391,0.961957
25%,0.983696,0.945652,0.971014,0.964674,0.945652,0.983696,0.971014,0.964674,0.945652,0.983696,0.971014,0.964674
50%,0.983696,0.967391,0.981884,0.978261,0.967391,0.983696,0.981884,0.978261,0.967391,0.983696,0.981884,0.978261
75%,0.989130,0.989130,0.985507,0.986413,0.989130,0.989130,0.985507,0.986413,0.989130,0.989130,0.985507,0.986413
max,0.989130,0.989130,0.985507,0.986413,0.989130,0.989130,0.985507,0.986413,0.989130,0.989130,0.985507,0.986413


## Grid Search

In [41]:
from sklearn.model_selection import GridSearchCV

search_params = {'n_estimators': [1500, 1600, 1700],
               'max_features': ['sqrt','log2',.2],
               'max_depth': [30, 40, 50],
               'min_samples_split': [4, 5, 6],
               'min_samples_leaf': [1, 2],
               'bootstrap': [False, True]}

rf_search_grid = GridSearchCV(rf, search_params, 
                          cv = 3, n_jobs = -1, verbose = 2)
rf_search_grid.fit(X_train, y_train)

Fitting 3 folds for each of 324 candidates, totalling 972 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 12 concurrent workers.
[Parallel(n_jobs=-1)]: Done  17 tasks      | elapsed:    9.5s
[Parallel(n_jobs=-1)]: Done 138 tasks      | elapsed:  1.0min
[Parallel(n_jobs=-1)]: Done 341 tasks      | elapsed:  2.5min
[Parallel(n_jobs=-1)]: Done 624 tasks      | elapsed:  5.0min
[Parallel(n_jobs=-1)]: Done 972 out of 972 | elapsed:  8.1min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=RandomForestClassifier(bootstrap=True, ccp_alpha=0.0,
                                              class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features=0.5,
                                              max_leaf_nodes=None,
                                              max_samples=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators=100, n_jobs=None,
                                              oob_score=False, random_state=555,
                                    

In [42]:
rf_search_grid.best_params_

{'bootstrap': False,
 'max_depth': 30,
 'max_features': 'log2',
 'min_samples_leaf': 2,
 'min_samples_split': 5,
 'n_estimators': 1500}

In [43]:
best = rf_search_grid.best_estimator_
df_grid = splitrepeat_cv(X,y,best,splits=[10,20,30],repeats=[10,20,30])
df_grid.describe()

,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000,9.000000
mean,0.984903,0.967391,0.979066,0.976147,0.967391,0.984903,0.979066,0.976147,0.967391,0.984903,0.979066,0.976147
std,0.003623,0.018827,0.007196,0.009964,0.018827,0.003623,0.007196,0.009964,0.018827,0.003623,0.007196,0.009964
min,0.978261,0.945652,0.967391,0.961957,0.945652,0.978261,0.967391,0.961957,0.945652,0.978261,0.967391,0.961957
25%,0.983696,0.945652,0.971014,0.964674,0.945652,0.983696,0.971014,0.964674,0.945652,0.983696,0.971014,0.964674
50%,0.983696,0.967391,0.981884,0.978261,0.967391,0.983696,0.981884,0.978261,0.967391,0.983696,0.981884,0.978261
75%,0.989130,0.989130,0.985507,0.986413,0.989130,0.989130,0.985507,0.986413,0.989130,0.989130,0.985507,0.986413
max,0.989130,0.989130,0.985507,0.986413,0.989130,0.989130,0.985507,0.986413,0.989130,0.989130,0.985507,0.986413


In [44]:
rf = RandomForestClassifier(n_estimators=1500, max_features='log2', max_depth = 30, min_samples_split = 5, min_samples_leaf = 2, bootstrap=False, random_state=33433)

%time \
df = splitrepeat_cv(X,y,rf,splits=[111,222,333,444],repeats=[111,222,333,444,555])
df.describe()

Wall time: 48.3 s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000,20.000000
mean,0.980163,0.975000,0.978442,0.977582,0.975000,0.980163,0.978442,0.977582,0.975000,0.980163,0.978442,0.977582
std,0.010017,0.018017,0.004152,0.005906,0.018017,0.010017,0.004152,0.005906,0.018017,0.010017,0.004152,0.005906
min,0.961957,0.945652,0.971014,0.967391,0.945652,0.961957,0.971014,0.967391,0.945652,0.961957,0.971014,0.967391
25%,0.976902,0.970109,0.974638,0.972147,0.970109,0.976902,0.974638,0.972147,0.970109,0.976902,0.974638,0.972147
50%,0.980978,0.978261,0.980072,0.980978,0.978261,0.980978,0.980072,0.980978,0.978261,0.980978,0.980072,0.980978
75%,0.985054,0.989130,0.981884,0.981658,0.989130,0.985054,0.981884,0.981658,0.989130,0.985054,0.981884,0.981658
max,0.994565,0.989130,0.981884,0.983696,0.989130,0.994565,0.981884,0.983696,0.989130,0.994565,0.981884,0.983696


In [2]:
# All samples
rf = RandomForestClassifier(n_estimators=1500, max_features='log2', max_depth = 30, min_samples_split = 5, min_samples_leaf = 2, bootstrap=False, random_state=33433)

%time \
df = splitrepeat_cv(X,y,rf,splits=list(int(x)*42+42 for x in range(10)),repeats=list(int(x)*42+42 for x in range(10)), imbalanced='over', categorical_features=categorical_features, initial_split_seed=33433, initial_split_ratio=.25)
df.describe()

Wall time: 7min 8s


,Sensitivity0,Specificity0,Accuracy0,AUC0,Sensitivity1,Specificity1,Accuracy1,AUC1,Sensitivity,Specificity,Accuracy,AUC
count,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000
mean,0.990830,0.643529,0.970748,0.817180,0.643529,0.990830,0.970748,0.817180,0.643529,0.990830,0.970748,0.817180
std,0.005392,0.159255,0.008954,0.078792,0.159255,0.005392,0.008954,0.078792,0.159255,0.005392,0.008954,0.078792
min,0.978339,0.294118,0.945578,0.639839,0.294118,0.978339,0.945578,0.639839,0.294118,0.978339,0.945578,0.639839
25%,0.985560,0.529412,0.965986,0.762450,0.529412,0.985560,0.965986,0.762450,0.529412,0.985560,0.965986,0.762450
50%,0.989170,0.705882,0.972789,0.845721,0.705882,0.989170,0.972789,0.845721,0.705882,0.989170,0.972789,0.845721
75%,0.996390,0.764706,0.979592,0.876938,0.764706,0.996390,0.979592,0.876938,0.764706,0.996390,0.979592,0.876938
max,1.000000,0.882353,0.982993,0.935761,0.882353,1.000000,0.982993,0.935761,0.882353,1.000000,0.982993,0.935761


# Outputs

In [5]:
### Save outputs ###
q = 'CDR1_AllFeatures_2'

df['set'] = q
df.to_csv(('../models/outputs/' + q + '.csv'), index=False)

OSError: [Errno 22] Invalid argument